# Data Preprocessing
---

## Data Preprocessing Tools


### Import Libraries

In [6]:
import numpy as np # for converting or minipulating the data
import matplotlib.pyplot as plt # for plotting the data on graph to see
import pandas as pd # for importing the data

There are more tools that we will use as we advance.\
These are the basic tools for now so we can focus more understanding what we are doing and why.\
In fact there are libraries that we will use to help in the preprocessing stage.\
More on that later.

### Import Dataset
Here we imported that Data.csv and stored it in Variable called dataset.\
What csv_read returns is called a dataframe.\
A Dataframe is a 2 dimensional data structure, like a 2 dimensional array.

In [7]:
dataset = pd.read_csv('Data.csv')

In any dataset, that you will train a model with, you will have two disticnt entities.\
The set a features, which are the known characteristics of the observation.\
Or the independent variable(s).\
Then the set of dependent variable.\
\
Lets take a look at our data set.

In [8]:
print(dataset)

   Country   Age   Salary Purchased
0   France  44.0  72000.0        No
1    Spain  27.0  48000.0       Yes
2  Germany  30.0  54000.0        No
3    Spain  38.0  61000.0        No
4  Germany  40.0      NaN       Yes
5   France  35.0  58000.0       Yes
6    Spain   NaN  52000.0        No
7   France  48.0  79000.0       Yes
8  Germany  50.0  83000.0        No
9   France  37.0  67000.0       Yes


From here 4 colums, location, age, income, and if the bought a product.\
Real quick we can make out that if a person purchased a product is less likey to affect the other three columns.\
While any of the other may affect if someone does buy that product.\
We can safely say that (Country, Age, Salary) are the features and (Purchased) is the independent variable.\
We also have two cells with missing data.  More on that later.

In [11]:
x = dataset.iloc[:,:-1] # All Rows, All Columns except the last / feature set / independent variables
y = dataset.iloc[:,-1] # All Rows, Only last column / output vecter / dependent variable

As already mentioned, dataset is a pandas dataframe that represents a tabel of data.\
Its not an array, but for now lets think of it as an array of arrays.\
We know in Python we index in a range like so: arr[ start : end : step ]\
We also know [ -1 ] is the same as indexing the last element.\
Well, what we might not know is that we can't index a dataframe like an array: dataset[ : ][ : -1 ]\
The Pandas dataframe frame has a built in method for indexing data called iloc.\
Pandas.iloc[  ] requires 1 argument but can take 2. indexing of the first dimension and then the second.\
Indexing values can bet both a range or a single index.

### Handle Missing Data

## Encoding Categorical Data

### Encoding The Independent Variable

### Encoding The Dependent Variable

---

## Split Data Into Train/Test Sets

---

## Feature Scaling